In [55]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re


from itertools import chain
from process_data import load_entities, load_task, vectorize, vectorize_kv_pairs, save_pickle, load_pickle, load_kv_pairs, lower_list


In [56]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

def load_task(fpath, max_length=None):
    with open (fpath, encoding='utf-8') as f:
        lines = f.readlines()
        data, story = [], []
        for l in lines:
            l = l.rstrip()
            turn, left = l.split(' ', 1)
            
            if turn == '1': # new story
                story = []

            if '\t' in left: # question
                q, a = left.split('\t', 1)
                q = tokenize(q)
                q = lower_list(q)
                if q[-1] == '?':
                    q = q[:-1]
                if '\t' in a:
                    a = a.split('\t')[0] # discard reward
                a = a.split('|') # may contain several labels
                a = lower_list(a)

                substory = [x for x in story if x]

                data.append((substory, q, a))
                story.append('')
            else: # normal sentence
                s = word_tokenize(left)
                if s[-1] == '.':
                    s = s[:-1]
                s = lower_list(s)
                story.append(s)

    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

# def vectorize(data, w2i, max_sentence_size, memory_size, entities=None):
def vectorize(data, w2i, story_maxlen, query_maxlen, entities=None):
    if entities:
        e2i = dict((e, i) for i, e in enumerate(entities))

    S, Q, A = [], [], []
    for story, question, answer in data:
        # Vectroize story
#         ss = []
#         for sentence in story:
#             s_pad_len = max(0, max_sentence_size - len(sentence))
#             ss.append([w2i[w] for w in sentence] + [0] * s_pad_len)

#         ss = ss[::-1][:memory_size] # discard old memory lager than memory_size(max_story_size)
#         # pad to memory_size
#         lm = max(0, memory_size - len(ss))
#         for _ in range(lm):
#             ss.append([0] * max_sentence_size)
        # Vectroize story
#         print('story;', story)
        s_pad_len = max(0, story_maxlen - len(story))
        s = [w2i[w] for w in story] + [0] * s_pad_len

        # Vectroize question
        q_pad_len = max(0, query_maxlen - len(question))
        q = [w2i[w] for w in question] + [0] * q_pad_len
        q = q[:query_maxlen]

        # Vectroize answer
        if entities:
            y = np.zeros(len(entities), dtype='byte')
            for a in answer:
                y[e2i[a]] = 1
        else:
            y = np.zeros(len(w2i) + 1) # +1 for nil word
            for a in answer:
                y[w2i[a]] = 1

        S.append(s)
        Q.append(q)
        A.append(y)
    
    S = np.array(S, dtype=np.uint16)
    Q = np.array(Q, dtype=np.uint16)
    A = np.array(A, dtype='byte')

    return S, Q, A

In [59]:
train_stories = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_train.txt')
test_stories = load_task('./data/tasks_1-20_v1-2/en/qa5_three-arg-relations_test.txt')

vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + answer)
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

w2i = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize(train_stories,
                                                               w2i,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize(test_stories,
                                                            w2i,
                                                            story_maxlen,
                                                            query_maxlen)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

/home/jonki/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


-
Vocab size: 40 unique words
Story max length: 525 words
Query max length: 7 words
Number of training stories: 1000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['bill', 'travelled', 'to', 'the', 'office', 'bill', 'picked', 'up', 'the', 'football', 'there', 'bill', 'went', 'to', 'the', 'bedroom', 'bill', 'gave', 'the', 'football', 'to', 'fred'], ['what', 'did', 'bill', 'give', 'to', 'fred'], ['football'])
-
Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (1000, 525)
inputs_test shape: (1000, 525)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (1000, 7)
queries_test shape: (1000, 7)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (1000, 40)
answers_test shape: (1000, 40)
-
Compiling...


In [60]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 1000 samples, validate on 1000 samples
Epoch 1/120
1000/1000 [==============================] - 0s - loss: 2.6458 - acc: 0.1580 - val_loss: 2.2266 - val_acc: 0.1430
Epoch 2/120
1000/1000 [==============================] - 0s - loss: 2.2424 - acc: 0.1420 - val_loss: 1.9998 - val_acc: 0.2830
Epoch 3/120
1000/1000 [==============================] - 0s - loss: 1.9039 - acc: 0.2480 - val_loss: 1.6111 - val_acc: 0.2830
Epoch 4/120
1000/1000 [==============================] - 0s - loss: 1.6456 - acc: 0.2800 - val_loss: 1.4897 - val_acc: 0.2840
Epoch 5/120
1000/1000 [==============================] - 0s - loss: 1.5271 - acc: 0.3190 - val_loss: 1.4782 - val_acc: 0.3010
Epoch 6/120
1000/1000 [==============================] - 0s - loss: 1.5110 - acc: 0.2820 - val_loss: 1.3804 - val_acc: 0.2910
Epoch 7/120
1000/1000 [==============================] - 0s - loss: 1.4643 - acc: 0.2740 - val_loss: 1.3630 - val_acc: 0.3010
Epoch 8/120
1000/1000 [==============================] - 0s - loss: 1.

1000/1000 [==============================] - 0s - loss: 0.4349 - acc: 0.8390 - val_loss: 2.0438 - val_acc: 0.3070
Epoch 66/120
1000/1000 [==============================] - 0s - loss: 0.4090 - acc: 0.8470 - val_loss: 2.1241 - val_acc: 0.3170
Epoch 67/120
1000/1000 [==============================] - 0s - loss: 0.4192 - acc: 0.8470 - val_loss: 2.1344 - val_acc: 0.3180
Epoch 68/120
1000/1000 [==============================] - 0s - loss: 0.4111 - acc: 0.8510 - val_loss: 2.1516 - val_acc: 0.3350
Epoch 69/120
1000/1000 [==============================] - 0s - loss: 0.4008 - acc: 0.8590 - val_loss: 2.1854 - val_acc: 0.3040
Epoch 70/120
1000/1000 [==============================] - 0s - loss: 0.4045 - acc: 0.8480 - val_loss: 2.1568 - val_acc: 0.3110
Epoch 71/120
1000/1000 [==============================] - 0s - loss: 0.3839 - acc: 0.8640 - val_loss: 2.1642 - val_acc: 0.3230
Epoch 72/120
1000/1000 [==============================] - 0s - loss: 0.3602 - acc: 0.8810 - val_loss: 2.1729 - val_acc: 0.33